In [1]:
import sys
sys.path.insert(0, '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version')
sys.path.insert(0, '/home/steve/Dokumenty/Studia/project/CSGNet')

from ucsgnet.dataset import CADDataset, get_simple_2d_transforms
import matplotlib.pyplot as plt
from ucsgnet.ucsgnet.cad.net_cad import Net
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import math
import argparse

In [2]:
params_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/models/pretrained/models/cad_main/initial/params.json'
weights_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/models/pretrained/models/cad_main/initial/ckpts/model.ckpt'
synthetic_dir = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/data/synthetic'
cad_path = '/home/steve/Dokumenty/Studia/project/ucsgnet-students-version/data/cad/cad.h5'

In [3]:
dataset = CADDataset(cad_path, '', get_simple_2d_transforms())
loader = DataLoader(
            dataset=dataset,
            batch_size=1, # 16
            shuffle=False,
            drop_last=False,
            num_workers=0,
        )
len(dataset), len(loader)

(3000, 3000)

In [17]:
import h5py
with h5py.File(cad_path, "r") as h5_file:
        _images = h5_file['train_images'][:]
_images.shape

(10000, 64, 64)

In [4]:
dataset[0][0].shape
# dataset[1][2]

torch.Size([1, 64, 64])

In [5]:
# hparams_dict = {
#     'use_planes': False,
#     'shapes_per_type': 16,
#     'num_dimensions': 2,
#     'out_shapes_per_layer': 4,
#     'weight_binarizing_threshold': 0.1,
#     'num_csg_layers': 2,
# }
# hparams = argparse.Namespace()
# for key, value in hparams_dict.items():
#     setattr(hparams, key, value)
# net = Net(hparams)
# # hparams
# net.net

Num of trainable params: 5709491
Num of not trainable params: 0


CSGNet(
  (encoder_): Extractor2D(
    (layers): Sequential(
      (0): Conv2DWithActivation(
        (conv): Conv2d(1, 32, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (1): Conv2DWithActivation(
        (conv): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (2): Conv2DWithActivation(
        (conv): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (3): Conv2DWithActivation(
        (conv): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
        (lrelu): LeakyReLU(negative_slope=0.01)
      )
      (4): Conv2DWithoutActivation(
        (conv): Conv2d(256, 256, kernel_size=(4, 4), stride=(2, 2))
      )
      (5): Flatten()
    )
  )
  (decoder_): Decoder(
    (layers): Sequential(
      (0): Linear(in_features=256, out_features=512, bias=Tru

In [6]:
net = Net.load_from_checkpoint(weights_path)
net.eval()
net.freeze()
net.net
net = Net(
    
)

In [7]:
prev = None
counter = {'t': 0, 'f': 0}
for batch in tqdm(loader):
    image1, points1, trues, _ = batch
    hidden = net.net.encoder_(image1)
    shape_params = net.net.decoder_(hidden)
    
    points = points1.unsqueeze(
        dim=1
    )  # broadcasting for different of shapes

    base_shapes = net.net.evaluator_(
        shape_params, points
    )  # -> batch, num_shapes, num_points

    base_shapes = base_shapes.permute(
        (0, 2, 1)
    )  # -> batch, num_points, num_shapes
    
    scaled_shapes = 1 - net.net.scaler_(base_shapes)
    pred = net(image1, points1)
    break
# counter

  0%|                                                                                                                                                                                                   | 0/3000 [00:00<?, ?it/s]

torch.Size([1, 4096, 48])
torch.Size([1, 4096, 48])


In [8]:
pred, trues, scaled_shapes.shape

(tensor([[0.5342, 0.5534, 0.5722,  ..., 0.9962, 0.9843, 0.9721]],
        grad_fn=<ClampBackward>),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([1, 4096, 48]))

In [9]:
from collections import Counter
Counter(scaled_shapes[...,0].tolist()[0])

Counter({0.31510722637176514: 1,
         0.3259986639022827: 1,
         0.3367035388946533: 1,
         0.3472127914428711: 1,
         0.357516884803772: 1,
         0.367605984210968: 1,
         0.3774697184562683: 1,
         0.38709747791290283: 1,
         0.3964780569076538: 1,
         0.40560001134872437: 1,
         0.41445136070251465: 1,
         0.4230199456214905: 1,
         0.4312931299209595: 1,
         0.43925803899765015: 1,
         0.4469016194343567: 1,
         0.4542105793952942: 1,
         0.4611715078353882: 1,
         0.467771053314209: 1,
         0.47399580478668213: 1,
         0.47983258962631226: 1,
         0.48526835441589355: 1,
         0.49029064178466797: 1,
         0.49488717317581177: 1,
         0.49904656410217285: 1,
         0.5027580261230469: 1,
         0.5060116052627563: 1,
         0.5087984800338745: 1,
         0.5111106634140015: 1,
         0.5129417181015015: 1,
         0.5142860412597656: 1,
         0.5151399374008179: 1,
